# Data preprocesing, Part 1

## Import modules and define utility functions

In [1]:
import cudf
import numpy as np
import pandas as pd
import gc
import pathlib
import gcsfs

In [2]:
def sizeof_fmt(num, suffix="B"):
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return "%.1f%s%s" % (num, "Yi", suffix)


def report_dataframe_size(df, name):
    print(
        "{} takes up {} memory on GPU".format(
            name, sizeof_fmt(grid_df.memory_usage(index=True).sum())
        )
    )

## Load Data

In [3]:
TARGET = "sales"  # Our main target
END_TRAIN = 1941  # Last day in train set

In [4]:
raw_data_dir = "./data/"
processed_data_dir = "./processed_data/"

pathlib.Path(processed_data_dir).mkdir(exist_ok=True)

In [5]:
train_df = cudf.read_csv(raw_data_dir + "sales_train_evaluation.csv")
prices_df = cudf.read_csv(raw_data_dir + "sell_prices.csv")
calendar_df = cudf.read_csv(raw_data_dir + "calendar.csv").rename(
    columns={"d": "day_id"}
)

In [6]:
train_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


The columns `d_1`, `d_2`, ..., `d_1941` indicate the sales data at days 1, 2, ..., 1941 from 2011-01-29.

In [7]:
prices_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [8]:
calendar_df

,date,wm_yr_wk,weekday,wday,month,year,day_id,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,<NA>,<NA>,<NA>,<NA>,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,<NA>,<NA>,<NA>,<NA>,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,<NA>,<NA>,<NA>,<NA>,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,<NA>,<NA>,<NA>,<NA>,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,<NA>,<NA>,<NA>,<NA>,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,<NA>,<NA>,<NA>,<NA>,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,<NA>,<NA>,<NA>,<NA>,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,<NA>,<NA>,<NA>,<NA>,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,<NA>,<NA>,<NA>,<NA>,0,0,0


## Reformat sales times series data

Pivot the columns `d_1`, `d_2`, ..., `d_1941` into separate rows using `cudf.melt`.

In [9]:
index_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]
grid_df = cudf.melt(
    train_df, id_vars=index_columns, var_name="day_id", value_name=TARGET
)
grid_df

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


For each time series, add 28 rows that corresponds to the future forecast horizon:

In [10]:
add_grid = cudf.DataFrame()
for i in range(1, 29):
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df["day_id"] = "d_" + str(END_TRAIN + i)
    temp_df[TARGET] = np.nan  # Sales amount at time (n + i) is unknown
    add_grid = cudf.concat([add_grid, temp_df])
add_grid["day_id"] = add_grid["day_id"].astype(
    "category"
)  # The day_id column is categorical, after cudf.melt

grid_df = cudf.concat([grid_df, add_grid])
grid_df = grid_df.reset_index(drop=True)
grid_df["sales"] = grid_df["sales"].astype(
    np.float32
)  # Use float32 type for sales column, to conserve memory
grid_df

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,NaN
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,NaN
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,NaN
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,NaN


### Free up GPU memory

GPU memory is a precious resource, so let's try to free up some memory. First, delete temporary variables we no longer need:

In [11]:
# Use xdel magic to scrub extra references from Jupyter notebook
%xdel temp_df
%xdel add_grid
%xdel train_df

# Invoke the garbage collector explicitly to free up memory
gc.collect()

8184

Second, let's reduce the footprint of `grid_df` by converting strings into categoricals:

In [12]:
report_dataframe_size(grid_df, "grid_df")

grid_df takes up 5.2GiB memory on GPU


In [13]:
grid_df.dtypes

id            object
item_id       object
dept_id       object
cat_id        object
store_id      object
state_id      object
day_id      category
sales        float32
dtype: object

In [14]:
for col in index_columns:
    grid_df[col] = grid_df[col].astype("category")
    gc.collect()
report_dataframe_size(grid_df, "grid_df")

grid_df takes up 802.6MiB memory on GPU


In [15]:
grid_df.dtypes

id          category
item_id     category
dept_id     category
cat_id      category
store_id    category
state_id    category
day_id      category
sales        float32
dtype: object

## Identify the release week of each product

Each row in the `prices_df` table contains the price of a product sold at a store for a given week.

In [16]:
prices_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


Notice that not all products were sold over every week. Some products were sold only during some weeks. Let's use the groupby operation to identify the first week in which each product went on the shelf.

In [17]:
release_df = (
    prices_df.groupby(["store_id", "item_id"])["wm_yr_wk"].agg("min").reset_index()
)
release_df.columns = ["store_id", "item_id", "release_week"]
release_df

,store_id,item_id,release_week
0,CA_4,FOODS_3_529,11421
1,TX_1,HOUSEHOLD_1_409,11230
2,WI_2,FOODS_3_145,11214
3,CA_4,HOUSEHOLD_1_494,11106
4,WI_3,HOBBIES_1_093,11223
...,...,...,...
30485,CA_3,HOUSEHOLD_1_369,11205
30486,CA_2,FOODS_3_109,11101
30487,CA_4,FOODS_2_119,11101
30488,CA_4,HOUSEHOLD_2_384,11110


Now that we've computed the release week for each product, let's merge it back to `grid_df`:

In [18]:
grid_df = grid_df.merge(release_df, on=["store_id", "item_id"], how="left")
grid_df = grid_df.sort_values(index_columns + ["day_id"]).reset_index(drop=True)
grid_df

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,release_week
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3.0,11101
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0.0,11101
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0.0,11101
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_4,1.0,11101
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_5,4.0,11101
...,...,...,...,...,...,...,...,...,...
60034805,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1965,NaN,11101
60034806,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1966,NaN,11101
60034807,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1967,NaN,11101
60034808,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_1968,NaN,11101


In [19]:
del release_df  # No longer needed
gc.collect()

138

In [20]:
report_dataframe_size(grid_df, "grid_df")

grid_df takes up 1.2GiB memory on GPU


## Filter out entries with zero sales

We can further save space by dropping rows from `grid_df` that correspond to zero sales. Since each product doesn't go on the shelf until its release week, its sale must be zero during any week that's prior to the release week.

To make use of this insight, we bring in the `wm_yr_wk` column from `calendar_df`:

In [21]:
grid_df = grid_df.merge(calendar_df[["wm_yr_wk", "day_id"]], on=["day_id"], how="left")
grid_df

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,release_week,wm_yr_wk
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1537,1.0,11101,11511
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1538,0.0,11101,11511
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1539,2.0,11101,11511
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1540,0.0,11101,11511
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1541,0.0,11101,11512
...,...,...,...,...,...,...,...,...,...,...
60034805,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_52,0.0,11101,11108
60034806,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_53,0.0,11101,11108
60034807,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_54,0.0,11101,11108
60034808,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_55,0.0,11101,11108


In [22]:
report_dataframe_size(grid_df, "grid_df")

grid_df takes up 1.7GiB memory on GPU


The `wm_yr_wk` column identifies the week that contains the day given by the `day_id` column. Now let's filter all rows in `grid_df` for which `wm_yr_wk` is less than `release_week`:

In [23]:
df = grid_df[grid_df["wm_yr_wk"] < grid_df["release_week"]]
df

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,release_week,wm_yr_wk
9990,FOODS_1_001_TX_3_evaluation,FOODS_1_001,FOODS_1,FOODS,TX_3,TX,d_1,0.0,11102,11101
9991,FOODS_1_001_TX_3_evaluation,FOODS_1_001,FOODS_1,FOODS,TX_3,TX,d_2,0.0,11102,11101
9992,FOODS_1_001_TX_3_evaluation,FOODS_1_001,FOODS_1,FOODS,TX_3,TX,d_3,0.0,11102,11101
9993,FOODS_1_001_TX_3_evaluation,FOODS_1_001,FOODS_1,FOODS,TX_3,TX,d_4,0.0,11102,11101
9994,FOODS_1_001_TX_3_evaluation,FOODS_1_001,FOODS_1,FOODS,TX_3,TX,d_5,0.0,11102,11101
...,...,...,...,...,...,...,...,...,...,...
60032955,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,d_20,0.0,11106,11103
60032956,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,d_21,0.0,11106,11103
60032957,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,d_22,0.0,11106,11104
60032958,HOUSEHOLD_2_516_WI_2_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,d_23,0.0,11106,11104


As we suspected, the sales amount is zero during weeks that come before the release week.

In [24]:
assert (df["sales"] == 0).all()

For the purpose of our data analysis, we can safely drop the rows with zero sales:

In [25]:
grid_df = grid_df[grid_df["wm_yr_wk"] >= grid_df["release_week"]].reset_index(drop=True)
grid_df["wm_yr_wk"] = grid_df["wm_yr_wk"].astype(
    np.int32
)  # Convert wm_yr_wk column to int32, to conserve memory
grid_df

,id,item_id,dept_id,cat_id,store_id,state_id,day_id,sales,release_week,wm_yr_wk
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1537,1.0,11101,11511
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1538,0.0,11101,11511
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1539,2.0,11101,11511
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1540,0.0,11101,11511
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1541,0.0,11101,11512
...,...,...,...,...,...,...,...,...,...,...
47735392,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_52,0.0,11101,11108
47735393,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_53,0.0,11101,11108
47735394,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_54,0.0,11101,11108
47735395,HOUSEHOLD_2_516_WI_3_evaluation,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,d_55,0.0,11101,11108


In [26]:
report_dataframe_size(grid_df, "grid_df")

grid_df takes up 1.2GiB memory on GPU


## Assign weights for product items

When we assess the accuracy of our machine learning model, we should assign a weight for each product item, to indicate the relative importance of the item. For the M5 competition, the weights are computed from the total sales amount (in US dollars) in the lastest 28 days.

In [27]:
# Convert day_id to integers
grid_df["day_id_int"] = grid_df["day_id"].to_pandas().apply(lambda x: x[2:]).astype(int)

# Compute the total sales over the latest 28 days, per product item
last28 = grid_df[(grid_df["day_id_int"] >= 1914) & (grid_df["day_id_int"] < 1942)]
last28 = last28[["item_id", "wm_yr_wk", "sales"]].merge(
    prices_df[["item_id", "wm_yr_wk", "sell_price"]], on=["item_id", "wm_yr_wk"]
)
last28["sales_usd"] = last28["sales"] * last28["sell_price"]
total_sales_usd = last28.groupby("item_id")[["sales_usd"]].agg(["sum"]).sort_index()
total_sales_usd.columns = total_sales_usd.columns.map("_".join)
total_sales_usd

,sales_usd_sum
item_id,
FOODS_1_001,3516.80
FOODS_1_002,12418.80
FOODS_1_003,5943.20
FOODS_1_004,54184.82
FOODS_1_005,17877.00
...,...
HOUSEHOLD_2_512,6034.40
HOUSEHOLD_2_513,2668.80
HOUSEHOLD_2_514,9574.60


To obtain weights, we normalize the sales amount for one item by the total sales for all items.

In [28]:
weights = total_sales_usd / total_sales_usd.sum()
weights = weights.rename(columns={"sales_usd_sum": "weights"})
weights

,weights
item_id,
FOODS_1_001,0.000090
FOODS_1_002,0.000318
FOODS_1_003,0.000152
FOODS_1_004,0.001389
FOODS_1_005,0.000458
...,...
HOUSEHOLD_2_512,0.000155
HOUSEHOLD_2_513,0.000068
HOUSEHOLD_2_514,0.000245


## Persist the processed data to disk

In [29]:
# No longer needed
del grid_df["day_id_int"]

# Persist grid_df to disk
grid_df.to_pandas().to_pickle(processed_data_dir + "grid_df_part1.pkl")
weights.to_pandas().to_pickle(processed_data_dir + "product_weights.pkl")